In [ ]:
import pandas as pd

In [ ]:
#data = pd.read_parquet("gs://ehh2022-challenge6/labs_before_or_without_ckd_diagnosis.parquet")
data = pd.read_parquet("gs://ehh2022-challenge6/training_set/labs_180days_before_ckd_diag_table.parquet")
#data = pd.read_parquet("gs://ehh2022-challenge6/training_set/labs_more_than_800_before_last_diag_table.parquet")

In [ ]:
# Drop data without values
data = data[~data.ValueNumber.isna()]

In [ ]:
# Take 300 most common metrics
tmp = data[~data.EntryDateTime.isna()]["metric_id"].value_counts()[:300]
top_codes = list(tmp.index)

In [ ]:
# Group more measurements in same time into one average
avg_data = data[data.metric_id.isin(top_codes)][["Patient", "EntryDateTime", "metric_id", "ValueNumber"]]
avg_data = avg_data.groupby(["Patient", "EntryDateTime", "metric_id"]).ValueNumber.mean().reset_index()

In [ ]:
# Pivot table
pivoted = final.pivot(index=["Patient", "EntryDateTime"], columns="metric_id", values="ValueNumber")

In [ ]:
# Final cleanup of obviously invalid values
pivoted[pivoted["Height"] > 350] = float("nan")
pivoted[pivoted["Height"] < 80] = float("nan")
pivoted[pivoted["Weight"] > 350] = float("nan")
pivoted[pivoted["Weight"] < 25] = float("nan")
pivoted = pivoted.drop(["BMI"], axis=1)

In [119]:
# Saving final product
pivoted.to_parquet("pivoted.pq")